In [100]:
import pandas as pd
import re
import json
from enum import Enum, auto

In [101]:
INPUT_DATASET_PATH = "data/input_dataset_v.1.xlsx"
UNITS_OF_MEASUREMENT_DATASET_PATH = "data/units_of_measurement.csv"
CONVERSION_RATES_PATH = "data/conversion_rates.csv"
UNITS_OF_MEASUREMENT_VARIATIONS_PATH = "data/units_of_measurement_variations.json"

class NumberNotations(Enum):
    EU = auto()
    US = auto()

In [102]:
with open(UNITS_OF_MEASUREMENT_VARIATIONS_PATH) as json_reader:
    unit_variations = json.load(json_reader)

In [ ]:
def standardize_units(text:str)->str:
    for variation, standard_unit in unit_variations.items():
        if variation in text:
            text = text.replace(variation,standard_unit)

    return text


def parse_amount_field(dataset:pd.DataFrame, field:str, number_format:NumberNotations=NumberNotations.US)->pd.DataFrame:
    if number_format == NumberNotations.US:
        dataset[field] = dataset[field].str.replace(',', '', regex=False)
    else:
        dataset[field] = dataset[field].str.replace('.', '', regex=False)
        dataset[field] = dataset[field].str.replace(',', '.', regex=False)
    
    dataset[field] = pd.to_numeric(dataset[field], errors='coerce')
    return dataset


def standardize_units_of_measurement(dataset:pd.DataFrame)->pd.DataFrame:
    df=pd.read_csv(CONVERSION_RATES_PATH)
    result = pd.merge(dataset,df, how='left')

    result['Unit'] = result['TargetUnit']
    result['Amount'] = result['Amount'] * result['ConversionRate']

    result = result.drop(['TargetUnit', 'ConversionRate'], axis=1)

    return result


def extract_description(dataset:pd.DataFrame)->pd.DataFrame:
    filtered_dataset = dataset.loc[dataset['GL_Account (General Ledger)'].isin(['Raw Materials Expense','Utilities Expense'])]/

In [104]:
cash_flow_dataset = pd.read_excel(INPUT_DATASET_PATH)
cash_flow_dataset = cash_flow_dataset.map(lambda s: s.lower() if type(s) == str else s)

units_of_measurment_dataset = pd.read_csv(UNITS_OF_MEASUREMENT_DATASET_PATH)
units_of_measurment_dataset = units_of_measurment_dataset.map(lambda s: s.lower() if type(s) == str else s)

In [105]:
cash_flow_dataset['Description'] = cash_flow_dataset['Description'].apply(standardize_units)

In [106]:
all_units='|'.join(list(units_of_measurment_dataset['Unit']))
pattern = rf'([\d.,]+)\s*({all_units})\b'

cash_flow_dataset[['Amount', 'Unit']] = cash_flow_dataset['Description'].str.extract(pattern, flags=re.IGNORECASE)

In [107]:
cash_flow_dataset = parse_amount_field(dataset=cash_flow_dataset, field='Amount')
cash_flow_dataset = parse_amount_field(dataset=cash_flow_dataset, field='Amount_EUR')

cash_flow_dataset = standardize_units_of_measurement(cash_flow_dataset)

cash_flow_dataset = cash_flow_dataset[['Description','Amount','Unit','Amount_EUR']]

In [ ]:
cash_flow_dataset

,Description,Amount,Unit,Amount_EUR
0,"invoice #45-a, agricultural diesel",NaN,NaN,1200.0
1,electricity bill - offices,NaN,NaN,350.0
2,payment for legal services,NaN,NaN,2500.0
3,"purchase ammonium nitrate 10,000t",10000000.0,kg,7000.0
4,electricity bill - irrigation pumps,NaN,NaN,1800.0
5,tractor t-100 maintenance,NaN,NaN,600.0
6,"invoice #88-b, diesel fuel",NaN,NaN,1500.0
7,office stationery purchase,NaN,NaN,150.0
8,herbicide purchase – 500l,500.0,l,NaN
9,water bill - irrigation system,NaN,NaN,NaN
